In [ ]:
# Complete LLM Integration Examples

This notebook provides complete, working examples of integrating the prompt system with various LLM providers.

## Overview

We'll cover:
- 🤖 **OpenAI Integration**: Chat completions with structured JSON
- 🧠 **Anthropic Integration**: Claude with response validation  
- ⚡ **Error Handling**: Robust error management patterns
- 🔄 **Response Processing**: Parsing and validation workflows
- 📊 **Real Examples**: Working code with actual disease analysis

**Note**: This notebook includes mock examples that you can adapt with your actual API keys.


In [ ]:
# Setup and imports
import json
import os
from typing import Dict, Any

# Initialize prompt system
import apps.research_prioritization.prompts.prompt_registry
from utils.prompts import Prompter, PromptNotFoundError
from pydantic import ValidationError

# Mock LLM responses for demonstration
MOCK_RESPONSES = {
    "socioeconomic": {
        "orphacode": "905",
        "disease_name": "Wilson disease",
        "socioeconomic_impact_studies": [
            {
                "cost": 25000,
                "measure": "Annual healthcare costs",
                "label": "Economic burden of Wilson disease in Spain",
                "source": "https://pubmed.ncbi.nlm.nih.gov/12345678",
                "country": "Spain",
                "year": "2023"
            }
        ],
        "score": 7,
        "evidence_level": "Medium-High evidence",
        "justification": "Based on peer-reviewed Spanish cost-of-illness study with comprehensive healthcare cost analysis."
    }
}

print("🚀 Setup complete!")
prompter = Prompter()
print(f"📋 Available prompts: {prompter.list_prompts()}")


In [ ]:
# Complete integration function (with mock LLM for demo)
def analyze_disease_with_prompts(
    disease_name: str, 
    orphacode: str, 
    prompt_type: str = "socioeconomic",
    version: str = "v1",
    use_mock: bool = True
):
    """
    Complete disease analysis using the prompt system.
    
    Args:
        disease_name: Name of the disease
        orphacode: ORPHA code  
        prompt_type: Type of analysis (socioeconomic, groups)
        version: Prompt version (v1, v2)
        use_mock: Use mock response for demo (True) or real LLM (False)
    """
    try:
        # 1. Get the prompt
        prompt_alias = f"{prompt_type}_{version}"
        prompt = prompter.get_prompt(prompt_alias)
        print(f"✅ Got prompt: {prompt_alias}")
        
        # 2. Format template
        template = prompt.template.format(
            disease_name=disease_name,
            orphacode=orphacode
        )
        print(f"📝 Template formatted ({len(template)} chars)")
        
        # 3. Call LLM (mock or real)
        if use_mock:
            # Use mock response for demonstration
            llm_response = json.dumps(MOCK_RESPONSES[prompt_type])
            print("🎭 Using mock LLM response")
        else:
            # This is where you'd call your actual LLM
            # llm_response = your_llm_call(template)
            llm_response = json.dumps(MOCK_RESPONSES[prompt_type])
            print("🤖 Would call real LLM here")
            
        # 4. Parse response
        parsed_response = prompt.parser(llm_response)
        print("🔄 Response parsed")
        
        # 5. Validate with Pydantic
        validated_data = prompt.model(**json.loads(parsed_response))
        print("✅ Data validated successfully!")
        
        return validated_data
        
    except PromptNotFoundError as e:
        print(f"❌ Prompt not found: {prompt_alias}")
        print(f"📋 Available: {e.available_prompts}")
        raise
        
    except ValidationError as e:
        print(f"❌ Validation failed:")
        for error in e.errors():
            print(f"   • {error['loc']}: {error['msg']}")
        raise
        
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        raise

# Test the function
result = analyze_disease_with_prompts(
    disease_name="Wilson disease",
    orphacode="905", 
    prompt_type="socioeconomic",
    version="v1"
)

print(f"\n🎯 Analysis Result:")
print(f"   Disease: {result.disease_name} ({result.orphacode})")
print(f"   Score: {result.score}")
print(f"   Evidence: {result.evidence_level}")
print(f"   Studies found: {len(result.socioeconomic_impact_studies)}")
